# Code Generator

The requirement: use a Frontier model to generate high performance C++ code from Python code


In [1]:
# imports

import os
import io
import sys
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess

In [2]:
# environment

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')

In [28]:
# initialize
# NOTE - option to use ultra-low cost models by uncommenting last 2 lines

openai = OpenAI()
# claude = anthropic.Anthropic()
OPENAI_MODEL = "gpt-4o"
# CLAUDE_MODEL = "claude-3-5-sonnet-20240620"

# Want to keep costs ultra-low? Uncomment these lines:
# OPENAI_MODEL = "gpt-4o-mini"
# CLAUDE_MODEL = "claude-3-haiku-20240307"

In [4]:
# system_message = "You are an assistant that reimplements Python code in high-performance C++ and Java for an M1 Mac. "
# system_message += "Respond only with C++ and Java code; use comments sparingly and do not provide any explanation other than occasional comments. "
#  system_message += "The C++ and Java responses need to produce identical output in the fastest possible time."


In [5]:
# def user_prompt_for(python):
#     user_prompt = "Rewrite this Python code in C++ and Java with the fastest possible implementation that produces identical output in the least time. "
#     user_prompt += "Respond only with C++ and Java code; do not explain your work other than a few comments. "
#     user_prompt += "Pay attention to number types to ensure no int overflows. Remember to #include all necessary C++ packages such as iomanip for C++, and import required packages for Java.\n\n"
#     user_prompt += python
#     return user_prompt


In [6]:
# def messages_for(python):
#     return [
#         {"role": "system", "content": system_message},  # Includes the updated system message with C++ and Java
#         {"role": "user", "content": user_prompt_for(python)}  # Calls the updated user prompt function for C++ and Java
#     ]


In [29]:
def write_output(code, file_name):
    """Write the generated code to a file."""
    with open(file_name, "w") as f:
        f.write(code)


In [30]:
def system_message_for(language):
    """Create a system message tailored for the requested language."""
    return (
        f"You are an assistant that reimplements Python code in high-performance {language.upper()} for an M1 Mac. "
        f"Respond only with {language.upper()} code; do not explain your work other than occasional comments. "
        "Pay attention to number types to ensure no overflows and include all necessary packages.\n\n"
    )

def user_prompt_for(python):
    """Generate the user prompt."""
    return (
        "Rewrite this Python code in the requested language with the fastest possible implementation that produces "
        "identical output in the least time. Use appropriate syntax for the language.\n\n" + python
    )

def messages_for(python, language):
    """Generate the messages for GPT."""
    return [
        {"role": "system", "content": system_message_for(language)},
        {"role": "user", "content": user_prompt_for(python)},
    ]
    

In [31]:
def optimize_gpt(python, language="cpp"):
    """Optimize the given Python code and generate C++ or Java output."""
    code = ""
    for chunk in stream_gpt(python, language):
        print(chunk, end="")  # Stream the output
        code = chunk  # Store the final code
    
    file_name = f"optimized.{language}"
    write_output(code, file_name)
    print(f"\nCode written to {file_name}.")

In [ ]:
# def optimize_claude(python):
#     result = claude.messages.stream(
#         model=CLAUDE_MODEL,
#         max_tokens=2000,
#         system=system_message,
#         messages=[{"role": "user", "content": user_prompt_for(python)}],
#     )
#     reply = ""
#     with result as stream:
#         for text in stream.text_stream:
#             reply += text
#             print(text, end="", flush=True)
#     write_output(reply)

In [32]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [10]:
exec(pi)

Result: 3.141592658589
Execution Time: 46.954224 seconds


In [ ]:
optimize_gpt(python_code, language="java")


In [ ]:
optimize_gpt(python_code, language="cpp")


In [ ]:
exec(pi)

# Compiling C++ and executing

You can use any platform now (Windows,Mac,Linux) i have added compatiblity in this

In [ ]:
import subprocess
import platform

def compile_and_run(language="cpp"):
    """Compile and run the generated code."""
    is_windows = platform.system() == "Windows"
    
    if language == "cpp":
        if is_windows:
            # Windows: Use g++ (requires MinGW or equivalent installed)
            compile_command = ["g++", "-O3", "-std=c++17", "-o", "optimized.exe", "optimized.cpp"]
            execute_command = ["optimized.exe"]
        else:
            # Non-Windows: Use clang++
            compile_command = [
                "clang++", "-O3", "-std=c++17", "-march=armv8.3-a", "-o", "optimized", "optimized.cpp"
            ]
            execute_command = ["./optimized"]
    elif language == "java":
        # Both Windows and non-Windows use the same Java commands
        compile_command = ["javac", "optimized.java"]
        execute_command = ["java", "optimized"]
    else:
        raise ValueError("Unsupported language. Choose 'cpp' or 'java'.")

    # Compile
    try:
        subprocess.run(compile_command, check=True, shell=is_windows)
        print(f"{language.upper()} compilation successful.")
    except subprocess.CalledProcessError as e:
        print(f"{language.upper()} compilation failed:\n{e}")
        return

    # Run
    try:
        output = subprocess.run(
            execute_command, capture_output=True, text=True, shell=is_windows
        )
        print(f"{language.upper()} execution output:\n{output.stdout}")
    except subprocess.CalledProcessError as e:
        print(f"{language.upper()} execution failed:\n{e.stderr}")


In [ ]:
compile_and_run(language="cpp")


In [ ]:
# optimize_claude(pi)

In [ ]:
# Repeat for Claude - again, use the right approach for your platform

# !clang++ -O3 -std=c++17 -march=armv8.3-a -o optimized optimized.cpp
# !./optimized

In [26]:
python_hard = """# Be careful to support large number sizes

def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value
        
def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum

# Parameters
n = 10000         # Number of random numbers
initial_seed = 42 # Initial seed for the LCG
min_val = -10     # Minimum value of random numbers
max_val = 10      # Maximum value of random numbers

# Timing the function
import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Subarray Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""

In [ ]:
exec(python_hard)

In [ ]:
# optimize_gpt(python_hard)

In [ ]:
# # Replace this with the right C++ compile + execute command for your platform

# !clang++ -O3 -std=c++17 -march=armv8.3-a -o optimized optimized.cpp
# !./optimized

In [ ]:
# optimize_claude(python_hard)

In [ ]:
# Replace this with the right C++ compile + execute command for your platform

# !clang++ -O3 -std=c++17 -march=armv8.3-a -o optimized optimized.cpp
# !./optimized

In [19]:
def stream_gpt(python, language="cpp"):
    """Stream the GPT output for the requested language."""
    if language not in ["cpp", "java"]:
        raise ValueError("Invalid language specified. Choose 'cpp' or 'java'.")
    
    # Stream response
    stream = openai.ChatCompletion.create(
        model=OPENAI_MODEL, messages=messages_for(python, language), stream=True
    )
    reply = ""
    code_block = f"```{language}\n"  # Detect code block for the language

    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        
        # Clean the streamed reply
        cleaned_reply = reply.replace(code_block, "").replace("```", "")
        yield cleaned_reply


In [20]:
# def stream_claude(python):
#     result = claude.messages.stream(
#         model=CLAUDE_MODEL,
#         max_tokens=2000,
#         system=system_message,
#         messages=[{"role": "user", "content": user_prompt_for(python)}],
#     )
#     reply = ""
#     with result as stream:
#         for text in stream.text_stream:
#             reply += text
#             yield reply.replace('```cpp\n','').replace('```','')

In [21]:
def optimize(python, model="GPT", language="cpp"):
    """
    Optimize the given Python code using the specified model (GPT or Claude) and generate the output
    in the requested programming language.

    Args:
        python (str): The Python code to optimize.
        model (str): The model to use ("GPT" or "Claude").
        language (str): The target programming language ("cpp" or "java").

    Yields:
        str: The streamed output of the generated code.
    """
    if model == "GPT":
        result = stream_gpt(python, language=language)
  
    else:
        raise ValueError("Unknown model. Please choose 'GPT' or 'Claude'.")

    for stream_so_far in result:
        yield stream_so_far


In [ ]:
# import gradio as gr

# # Assuming `optimize` is already defined and imported
# # python_hard should be a pre-defined Python code snippet for the default value

# with gr.Blocks() as ui:
#     with gr.Row():
#         python = gr.Textbox(label="Python code:", lines=10, value="")  # Default value can be set here
#         cpp = gr.Textbox(label="Converted code:", lines=10, interactive=False)  # Output box

#     with gr.Row():
#         model = gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT")  # Default is GPT
#         language = gr.Dropdown(["cpp", "java"], label="Target language", value="cpp")  # Default is C++
#         convert = gr.Button("Convert code")

#     # Connect the button to the optimize function
#     def convert_code(python, model, language):
#         result = ""
#         for output in optimize(python, model=model, language=language):
#             result = output  # Collect the last streamed result
#         return result

#     convert.click(
#         fn=convert_code,
#         inputs=[python, model, language],  # Inputs from UI
#         outputs=[cpp],  # Output to the C++/Java box
#     )

# ui.launch(inbrowser=True)


In [22]:
import io
import sys

def execute_python(code):
    """
    Execute Python code dynamically and capture its output.

    Args:
        code (str): The Python code to execute.

    Returns:
        str: The captured standard output of the executed code.

    Raises:
        Exception: If the execution of the code raises an error.
    """
    output = io.StringIO()
    try:
        sys.stdout = output  # Redirect standard output to the StringIO object
        exec(code, {})  # Execute code with an empty global context for safety
    except Exception as e:
        return f"Error during execution: {str(e)}"
    finally:
        sys.stdout = sys.__stdout__  # Restore standard output

    return output.getvalue()


In [23]:
# You'll need to change the code in the try block to compile the C++ code for your platform
# I pasted this into Claude's chat UI with a request for it to give me a version for an Intel PC,
# and it responded with something that looks perfect - you can try a similar approach for your platform.

# M1 Mac version to compile and execute optimized C++ code:

def execute_cpp(code):
        write_output(code)
        try:
            compile_cmd = ["clang++", "-Ofast", "-std=c++17", "-march=armv8.5-a", "-mtune=apple-m1", "-mcpu=apple-m1", "-o", "optimized", "optimized.cpp"]
            compile_result = subprocess.run(compile_cmd, check=True, text=True, capture_output=True)
            run_cmd = ["./optimized"]
            run_result = subprocess.run(run_cmd, check=True, text=True, capture_output=True)
            return run_result.stdout
        except subprocess.CalledProcessError as e:
            return f"An error occurred:\n{e.stderr}"

In [36]:

def execute_java(code):
    """Compile and execute Java code dynamically."""
    write_output(code, "Optimized.java")
    try:
        # Compile the Java code
        compile_cmd = ["javac", "Optimized.java"]
        subprocess.run(compile_cmd, check=True, text=True, capture_output=True)
        
        # Run the compiled Java program
        run_cmd = ["java", "Optimized"]
        run_result = subprocess.run(run_cmd, check=True, text=True, capture_output=True)
        return run_result.stdout  # Return the output
    except subprocess.CalledProcessError as e:
        return f"Error during compilation or execution:\n{e.stderr}"

In [24]:
# css = """
# .python {background-color: #306998;}
# .cpp {background-color: #050;}
# """

In [40]:
css = """
.python {background-color: #306998;}
.cpp {background-color: #050;}
.java {background-color: #b07219;}
"""

with gr.Blocks(css=css) as ui:
    gr.Markdown("## Convert code from Python to C++, Java, or Run Directly")
    
    with gr.Row():
        python = gr.Textbox(label="Python code:", value="print('Hello from Python!')", lines=10)
        cpp = gr.Textbox(label="C++ code:", lines=10)
        java = gr.Textbox(label="Java code:", lines=10)
    
    with gr.Row():
        model = gr.Dropdown(["GPT"], label="Select model", value="GPT")
    
    with gr.Row():
        convert_cpp = gr.Button("Convert to C++")
        convert_java = gr.Button("Convert to Java")
    
    with gr.Row():
        python_run = gr.Button("Run Python")
        cpp_run = gr.Button("Run C++")
        java_run = gr.Button("Run Java")
    
    with gr.Row():
        python_out = gr.TextArea(label="Python result:", elem_classes=["python"])
        cpp_out = gr.TextArea(label="C++ result:", elem_classes=["cpp"])
        java_out = gr.TextArea(label="Java result:", elem_classes=["java"])

    # Add C++ and Java conversion
    convert_cpp.click(optimize, inputs=[python, model], outputs=[cpp])
    convert_java.click(optimize, inputs=[python, model], outputs=[java])
    
    # Add execution buttons for each language
    python_run.click(execute_python, inputs=[python], outputs=[python_out])
    cpp_run.click(execute_cpp, inputs=[cpp], outputs=[cpp_out])
    java_run.click(execute_java, inputs=[java], outputs=[java_out])

ui.launch(inbrowser=True)
